In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
import os
from torchsummary import summary
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae


# Code used to process the data and create a training and testing dataset

We don't need this code for running the model and needed to run it just once for creating the pickled data files. So, we have commented it out and included it here

In [ ]:
# d = {"Jan": "01", "Feb": "02", "Mar": "03", "Apr": "04", "May": "05", "Jun": "06", "Jul": "07", "Aug": "08", "Sep": "09", "Oct": "10", "Nov": "11", "Dec":"12"}
# def date_transform(date):
#     month, year = date.split(" ")
#     month = d[month]
#     res = int(year + month)
#     return res

# inflation_df = pd.read_excel(f"{project_dir}/inflation.xlsx", sheet_name="fcpi_m", index_col=0)
# us_fcpi = inflation_df.loc["USA"]                                                               # monthly food cpi Series

# ppd_df = pd.read_csv(f"{project_dir}/ppd.csv")
# ppd_df["Month/Year"]= ppd_df["Month/Year"].apply(date_transform)
# ppd_df = ppd_df[ppd_df["Month/Year"] < 202203]                                                  # since the inflation data doenst not have 202203 onwards

# # combine fcpi into ppd_df
# ppd_df["fcpi"] = ppd_df["Month/Year"].apply(lambda x : us_fcpi[x])
# ppd_df["Container or Bulk"] = np.where(ppd_df["Container or Bulk"] == "Bulk", 1, 0)
# ppd_df["Refrigerated or Dry"] = np.where(ppd_df["Refrigerated or Dry"],1, 0)
# ppd_df['Metric Tons'] *= ppd_df['Export or Import'].apply(lambda x: -1 if x=='Import' else 1) #metric tons value +ve if export and -ve if import
# ppd_df.drop(columns=['Export or Import', 'Year', 'Month'], inplace=True)
# ppd_df.to_pickle(f"{project_dir}/ppd.pkl")


# debug=ppd_df.sample(frac=0.05,random_state=1)
# train=ppd_df.drop(debug.index)
# debug.to_pickle(f"{project_dir}/debug.pkl")
# train.to_pickle(f"{project_dir}/train.pkl")

# Reading Data

In [ ]:
def getfile(location_pair,**kwargs):                                                                        #tries to get local version and then defaults to google drive version
    gdrive=location_pair
    loc = 'https://drive.google.com/uc?export=download&id='+gdrive.split('/')[-2]
    out=pd.read_pickle(loc,**kwargs)
    return out

                                                                                                            # Read the data from the dataset file
fname=("https://drive.google.com/file/d/1PDrNKb-vcb6YH1o18EvvYEXf2a82_DJq/view?usp=share_link")
ppd_df=getfile(fname)

Creating a mapping from port name to port ID

In [ ]:
port2num, num2port = dict(), dict()
num = 0
for port in ppd_df["Port"]:
  if port in port2num: continue
  port2num[port] = num
  num2port[num] = port
  num += 1

# Our baseline Linear Regression Model:
It only takes care of the numerical features, and the MSE we got is 1143.94 for the Metric Tons data (for the readability of the loss, we divide the label by 10000), which will be the baseline performance of the model. We will add categorial data with one-hot encoding later on in our Deep learning model.

In [ ]:
dev = getfile(fname)
dev["Port"] = dev["Port"].apply(lambda x : port2num[x])
features = ["TEU", "fcpi"]                                                                              #Extracting relevant features
X = dev[features]
y = dev["Metric Tons"] / 10000
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)               #Splitting dataset into 20% for test set and 80% for train set
losses = []

model = LinearRegression()
model.fit(X_train, y_train)
pred = model.predict(X_test)
loss = mse(pred, y_test)                                                                                #Calculating loss
print(f"Average MSE is {loss}")

Average MSE is 1143.944210921282


# Deep Learning Model
We have used an embedding layer for each categorical feature and then a linear layer to learn the important features. We finally combine the categorical and numerical features and run them through a dense network consisting of linear layers, a non-linearity and batch normalization layers.

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn
import torch
import torch.optim as optim

y = dev["Metric Tons"] / 10000
X = dev.loc[:, ~dev.columns.isin(['Metric Tons'])]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)                      #Splitting into 20% test set and 80% train set

class PortData(Dataset):                                                                                       #Class for the datasets
  def __init__(self, X, y):
    self.X1 = X[["TEU", "fcpi"]].values
    self.X2 = X[["Port", "Container or Bulk", "Refrigerated or Dry"]].values
    self.labels = y
    
  def __len__(self):
    return len(self.labels)
  
  def __getitem__(self, idx):
    return self.X1[idx], self.X2[idx], self.labels.iloc[idx]

device = "cuda" if torch.cuda.is_available() else "cpu"
train_dataset = PortData(X_train, y_train)
test_dataset = PortData(X_test, y_test)


In [ ]:
ports_cnt = len(port2num)
class Network(nn.Module):
  def __init__(self):
    super(Network, self).__init__()
    self.port_embed = nn.Embedding(ports_cnt, 64)
    self.port_dense = nn.Linear(64, 64)
    self.bulk_embed = nn.Embedding(2, 3)
    self.bulk_dense = nn.Linear(3, 32)
    self.refri_embed = nn.Embedding(2, 3)
    self.refri_dense = nn.Linear(3, 32)


    self.full_dense = nn.Sequential(                                                               #Dense layer (mini batches greater than 4) after combining categorical and numerical features
        nn.Linear(130, 256),
        nn.BatchNorm1d(256),
        nn.LeakyReLU(),
        nn.Linear(256, 128),
        nn.LeakyReLU(),
        nn.Linear(128, 16),
        nn.LeakyReLU(),
        nn.Linear(16, 1)
    )

    self.full_densebs1 = nn.Sequential(                                                           #Dense layer (mini batches lesser than or equal to 4) after combining categorical and numerical features
        nn.Linear(130, 256),
        nn.LeakyReLU(),
        nn.Linear(256, 128),
        nn.LeakyReLU(),
        nn.Linear(128, 16),
        nn.LeakyReLU(),
        nn.Linear(16, 1)
    )
  
  def forward(self, X_num, X_cat):                                                                 #Forward pass
    port_data = F.leaky_relu(self.port_embed(X_cat[:,0]))
    port_data = F.leaky_relu(self.port_dense(port_data))
    bulk_data= F.leaky_relu(self.bulk_embed(X_cat[:, 1]))
    bulk_data = F.leaky_relu(self.bulk_dense(bulk_data))
    refri_data = F.leaky_relu(self.refri_embed(X_cat[:,2]))
    refri_data = F.leaky_relu(self.refri_dense(refri_data))

    all_combined = torch.cat((port_data, bulk_data, refri_data, X_num), 1)
    return self.full_dense(all_combined)



def getOptim(m, lr=0.001, momen=0.1, op="Adam"):                                                    #Chossing an optimizer
  if op == "Adam":
    return optim.Adam(m.parameters(), lr = lr)
  elif op == "SGD":
    return optim.SGD(m.parameters(), lr = lr, momentum = momen)
  elif op == "RMS":
    return optim.RMSprop(m.parameters(), lr = lr, momentum = momen)


def get_loader(batch_size = 32, stage = "train"):                                                   #Loading the data (pass the appropriate batch size into this function for mini batch GD)
  if (stage == "train"):
    return DataLoader(train_dataset, batch_size = batch_size)
  return DataLoader(test_dataset, batch_size = batch_size)


def train_minibatch(m, epoch=100, op="Adam", lr=0.001, momen=0.1, batch_size=32):                   #Trainer used to compare different batch sizes for mini batch GD based on train and test error (Not actually
                                                                                                    #used for a single run of the model)
  for bs in [31955,512,256,128,64,32,16,8,4]:
      loss = nn.MSELoss()
      optim = getOptim(m,lr=lr, momen=momen, op=op)
      train_loader = get_loader(batch_size=bs)
      for e in range(epoch):
        curLoss = 0
        for i, data in enumerate(train_loader):
          X_num = data[0].float().to(device)
          X_cat = data[1].to(device)
          label = data[2].float().to(device)
          optim.zero_grad()
          if bs != 1:
            outputs = m(X_num, X_cat).squeeze()
          else:
            outputs = m(X_num, X_cat)
          l = loss(outputs, label)
          l.backward()
          optim.step()
          curLoss += l.item()
        if e == 99:
          print(f'{bs} batch size average Loss is: {curLoss / len(train_loader)}')
      
      
      test_loader = get_loader(bs, "test")
      all_loss=0

      with torch.no_grad():
        loss = nn.MSELoss()
        for data in test_loader:
          X_num = data[0].float().to(device)
          X_cat = data[1].to(device)
          label = data[2].float().to(device)
          outputs = model(X_num, X_cat).squeeze()
          all_loss += (loss(outputs, label) * data[0].size(0))

      mse = all_loss / len(test_dataset)
      print(f"the mse of testing set is {mse}")


def train(m, epoch=100, op="Adam", lr=0.001, momen=0.1, batch_size=32):                             #Trainer actually used for a single run with the best parameters
  loss = nn.MSELoss()
  optim = getOptim(m,lr=lr, momen=momen, op=op)
  train_loader = get_loader(batch_size=batch_size)
  for e in range(epoch):
    curLoss = 0
    for i, data in enumerate(train_loader):
      X_num = data[0].float().to(device)                                                            #Separates the categorical and numerical features for separate handling
      X_cat = data[1].to(device)
      label = data[2].float().to(device)
      optim.zero_grad()
      outputs = m(X_num, X_cat).squeeze()
      l = loss(outputs, label)
      l.backward()
      optim.step()
      curLoss += l.item()
    if e % 10 == 0:
      print(f'{e}th epoch average Loss is: {curLoss / len(train_loader)}')                          #Printing the loss in the model

model = Network().to(device)
train(model,batch_size = 128)

0th epoch average Loss is: 1279.5553597564697
10th epoch average Loss is: 1062.7306218719482
20th epoch average Loss is: 1057.183671356201
30th epoch average Loss is: 1048.4794489746093
40th epoch average Loss is: 1046.6051750106813
50th epoch average Loss is: 1042.7283987960816
60th epoch average Loss is: 1040.1866698608399
70th epoch average Loss is: 1032.7182002792358
80th epoch average Loss is: 1025.8584277877808
90th epoch average Loss is: 1016.3845274734497


Using SGD as the optimiser gave worse results (despite tuning learning rate and momentum) for the same architecture (as compared to Adam with all other parameters and hyperparameters constant). The 100th epoch average loss was in the range of 1100-1200 as compared to 600-700 for Adam. For RMS, it was found to be in the range 700-800.

Also, a batch size of 128 gave the best results for test set error (around 800) compared to other batch sizes or using the entire dataset (batch size = 31955) which gave test set errors of 900 to 1600

# Testing the model on the Test Set
Running the model we trained above on the test set

In [9]:
batchSize = 128
test_loader = get_loader(batchSize, "test")
all_loss=0

with torch.no_grad():
  loss = nn.MSELoss()
  for data in test_loader:
    X_num = data[0].float().to(device)
    X_cat = data[1].to(device)
    label = data[2].float().to(device)
    outputs = model(X_num, X_cat).squeeze()
    all_loss += (loss(outputs, label) * data[0].size(0))

mse = all_loss / len(test_dataset)
print(f"the mse of testing set is {mse}")

the mse of testing set is 890.8041381835938


# Saving the model
Note: Before running this block, please change project_dir to a location in your google drive where you want to store the trained model

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


project_dir = "/content/gdrive/MyDrive/CS547"
torch.save(model.state_dict, f'{project_dir}/net.pth')